<a href="https://colab.research.google.com/github/Anushkapandey2110/Fish_Freshness_Model/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#connecting to google drive
from google.colab import drive
drive.mount('/content/gdrive')
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
import os

# Define the paths for train, test, and validation datasets
base_dir = '/content/gdrive/MyDrive/Fish Eye Freshness.v2i.folder (1)'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
val_dir = os.path.join(base_dir, 'valid')

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

# Load the datasets
train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

validation_dataset = image_dataset_from_directory(
    val_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=False,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)
# Extract class names from the dataset
class_names = train_dataset.class_names

# Function to preprocess images and extract labels
def preprocess_data(images, labels):
    images = preprocess_input(images)
    labels = tf.one_hot(labels, depth=len(class_names))  # Convert labels to one-hot encoding
    return images, labels

# Preprocess and prepare datasets
train_dataset = train_dataset.map(preprocess_data)
validation_dataset = validation_dataset.map(preprocess_data)
test_dataset = test_dataset.map(preprocess_data)

# Define the ANN model
model = models.Sequential([
    layers.Flatten(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset,
                    validation_data=validation_dataset,
                    epochs=10)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test Accuracy: {test_accuracy:.4f}')

# Predictions
test_predictions = np.argmax(model.predict(test_dataset), axis=-1)
test_labels = np.concatenate([y for x, y in test_dataset], axis=0)
test_labels = np.argmax(test_labels, axis=-1)

# Calculate additional metrics
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')

print(f'Test Precision: {test_precision:.4f}')
print(f'Test Recall: {test_recall:.4f}')
print(f'Test F1 Score: {test_f1:.4f}')

# Classification Report
print("\nTest Classification Report:\n", classification_report(test_labels, test_predictions, target_names=class_names))


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Found 1080 files belonging to 2 classes.
Found 102 files belonging to 2 classes.
Found 51 files belonging to 2 classes.
Epoch 1/10
34/34 [==============================] - 153s 3s/step - loss: 35.3442 - accuracy: 0.5519 - val_loss: 1.1728 - val_accuracy: 0.6863
Epoch 2/10
34/34 [==============================] - 22s 620ms/step - loss: 9.9959 - accuracy: 0.5917 - val_loss: 1.2539 - val_accuracy: 0.5784
Epoch 3/10
34/34 [==============================] - 21s 565ms/step - loss: 7.0704 - accuracy: 0.5824 - val_loss: 0.6551 - val_accuracy: 0.7353
Epoch 4/10
34/34 [==============================] - 33s 917ms/step - loss: 4.4126 - accuracy: 0.6148 - val_loss: 1.1129 - val_accuracy: 0.6765
Epoch 5/10
34/34 [==============================] - 24s 633ms/step - loss: 3.3231 - accuracy: 0.6278 - val_loss: 0.8035 - val_accuracy: 0.6961
Epoch 6/10
34/34 [=================